In [ ]:
from datetime import datetime, timedelta
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

from pysida.lib import get_deformation_from_pair, get_triangulation, Pair

In [ ]:
# Converting database to pandas DataFrame
"""
with read_engine.connect() as conn:
    conn.execute(text("SET TRANSACTION ISOLATION LEVEL READ COMMITTED"))

    # Read only the filtered points from the database
    points = gpd.read_postgis(
        sql_query,
        conn,
        geom_col='geometry'
    )

# 'points' contains only the data for trajectories with more than 1 point
points_df = pd.DataFrame(points.drop(columns=['geometry', 'descriptors']))
points_df['x'] = points.geometry.x
points_df['y'] = points.geometry.y

points_df.to_feather('points.feather')
"""

In [ ]:
class GetRgpsPairsIm2:
    def __init__(self, df, max_time_diff, min_time_diff, min_size, r_min, a_max):
        self.df = df
        self.max_time_diff = max_time_diff
        self.min_time_diff = min_time_diff
        self.min_size = min_size
        self.r_min = r_min
        self.a_max = a_max

    def __call__(self, im2):
        pairs = []
        df2 = self.df[self.df.i == im2]
        d2 = df2.iloc[0].d

        gpi = (
            (self.df.d >= (d2 - timedelta(self.max_time_diff))) *
            (self.df.d <= (d2 - timedelta(self.min_time_diff)))
        )
        im1_idx = np.unique(self.df[gpi].i)
        for im1 in im1_idx:
            df1 = self.df[self.df.i == im1]
            _, i1, i2 = np.intersect1d(df1.g, df2.g, return_indices=True)
            if i1.size > self.min_size:

                x0 = df1.iloc[i1].x.to_numpy()
                x1 = df2.iloc[i2].x.to_numpy()
                y0 = df1.iloc[i1].y.to_numpy()
                y1 = df2.iloc[i2].y.to_numpy()
                q0 = df1.iloc[i1].q.to_numpy()
                q1 = df2.iloc[i2].q.to_numpy()

                t, a, p = get_triangulation(x0, y0)
                r = np.sqrt(a) / p
                g = (r >= self.r_min) * (a <= self.a_max)
                if g[g].size == 0:
                    continue

                pairs.append(Pair(
                    x0 = x0.astype(np.float32),
                    x1 = x1.astype(np.float32),
                    y0 = y0.astype(np.float32),
                    y1 = y1.astype(np.float32),
                    d0 = df1.iloc[i1[0]].d,
                    d1 = df2.iloc[i2[0]].d,
                    t = t,
                    a = a.astype(np.float32),
                    p = p.astype(np.float32),
                    g = g,
                    q0 = q0.astype(np.float32),
                    q1 = q1.astype(np.float32),
                ))
        return pairs

In [ ]:
points = pd.read_feather('points.feather')

In [ ]:
df = points[['image_id', 'trajectory_id', 'time', 'x', 'y', 'corr']].copy().rename(columns={'image_id':'i', 'trajectory_id': 'g', 'time':'d', 'x':'x', 'y':'y', 'corr':'q'})

In [ ]:
date0 = datetime(2020,1,1)
date1 = datetime(2020,1,2)

dfd = df[(df.d >= date0) * (df.d <= date1)]
im2_idx = np.unique(dfd.i)
print(len(im2_idx), 'images in the date range')
min_time_diff = 0.1
max_time_diff = 7
min_size = 10
r_min = 0.1
a_max = 500e6



In [ ]:
get_rgps_pairs_im2 = GetRgpsPairsIm2(df, max_time_diff, min_time_diff, min_size, r_min, a_max)
pairs = list(map(get_rgps_pairs_im2, im2_idx))
pairs = [j for i in pairs for j in i]

In [ ]:
pair_sizes = [p.x0.size for p in pairs]

In [ ]:
defors = [get_deformation_from_pair(p) for p in pairs]

In [ ]:
for p, d in zip(pairs, defors):
    if p.x0.size < 1000:
        continue
    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    trp0 = axs[0].tripcolor(p.x0, p.y0, d.e2, triangles=p.t, mask=~p.g, cmap='plasma_r', clim=[0, 0.0001])
    plt.colorbar(trp0, ax=axs[0], shrink=0.5)
    u, v = p.x1 - p.x0, p.y1 - p.y0
    qui1 = axs[1].quiver(p.x0, p.y0, u, v, p.q1, angles='xy', scale_units='xy', scale=0.1, cmap='jet', clim=[0,1])
    plt.colorbar(qui1, ax=axs[1], shrink=0.5)
    for ax in axs:
        ax.set_aspect('equal')
        ax.set_title(f'{p.d0} - {p.d1}')
    plt.show()
